In [1]:
!pip install -q flwr[simulation]

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 219.2/219.2 kB 3.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.1/2.1 MB 43.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 56.9/56.9 MB 27.7 MB/s eta 0:00:00


In [2]:
import flwr as fl
from flwr.common import Metrics
from flwr.common.typing import NDArrays, Scalar
from collections import OrderedDict
from typing import List, Tuple, Dict, Optional

import matplotlib.pyplot as plt
import numpy as np
import pandas as pd
import tensorflow as tf

import os
import librosa

In [3]:
from google.colab import drive
drive.mount('/content/gdrive')#, force_remount=True)

Mounted at /content/gdrive


In [4]:
%cd /content/gdrive/MyDrive/daic/ProjectPrototype

/content/gdrive/MyDrive/daic/ProjectPrototype


In [5]:
!ls

'212148conf copy 3.ipynb'   edaicwoz   MFCCs_1030   MFCCs_1030.zip   preprocess_data.ipynb


In [6]:
train_labels_df = pd.read_csv("edaicwoz/train_split.csv")
test_labels_df = pd.read_csv("edaicwoz/test_split.csv")
val_labels_df = pd.read_csv("edaicwoz/dev_split.csv")

In [7]:
def load_audio_files(data_dir, sr=16000):
    file_ids = os.listdir(data_dir)
    subject_ids = []
    file_paths = []
    types = []
    labels = []
    labels_binary = []

    for file_id in file_ids:
        file_id = file_id.split("_")[0]
        file_path = [data_dir + "/" + file_id + "/" + file_id + "_MFCC_" + str(i) + ".npy" for i in range(len(next(iter(enumerate(os.walk(data_dir + "/" + str(file_id) + "/"))))[1][2]))]
        if int(file_id) in train_labels_df["Participant_ID"].values:
            types.append(0)
            labels.append(train_labels_df[train_labels_df["Participant_ID"] == int(file_id)]['PHQ_Score'].values[0])
            labels_binary.append(train_labels_df[train_labels_df["Participant_ID"] == int(file_id)]['PHQ_Binary'].values[0])

        elif int(file_id) in test_labels_df["Participant_ID"].values:
            types.append(1)
            labels.append(test_labels_df[test_labels_df["Participant_ID"] == int(file_id)]['PHQ_Score'].values[0])
            labels_binary.append(test_labels_df[test_labels_df["Participant_ID"] == int(file_id)]['PHQ_Binary'].values[0])
        else:
            types.append(2)
            labels.append(val_labels_df[val_labels_df["Participant_ID"] == int(file_id)]['PHQ_Score'].values[0])
            labels_binary.append(val_labels_df[val_labels_df["Participant_ID"] == int(file_id)]['PHQ_Binary'].values[0])
        subject_ids.append(int(file_id))
        file_paths.append(file_path)

    return file_ids, subject_ids, file_paths, types, labels, labels_binary

data_dir = "MFCCs_1030"

file_ids, subject_ids, file_paths, types, labels, labels_binary = load_audio_files(data_dir)

In [8]:
def prepare_audio_set(file_paths):

    samples = []
    samples_ids = []
    samples_types = []
    samples_labels = []
    samples_labels_binary = []

    for i, file_path in enumerate(file_paths):
        all_mfccs = []
        for j in range(len(file_path)):
            all_mfccs.append(np.load(file_path[j]))
        all_mfccs = np.array(all_mfccs)
        samples.extend(all_mfccs)
        samples_ids.extend([subject_ids[i]] * len(all_mfccs))
        samples_types.extend([types[i]] * len(all_mfccs))
        samples_labels.extend([labels[i]] * len(all_mfccs))
        samples_labels_binary.extend([labels_binary[i]] * len(all_mfccs))

    samples = np.array(samples)

    samples_ids = np.array(samples_ids)
    samples_types = np.array(samples_types)
    samples_labels = np.array(samples_labels)
    samples_labels_binary = np.array(samples_labels_binary)

    return samples, samples_ids, samples_types, samples_labels, samples_labels_binary

print("[INFO] preparing data...")
samples, samples_ids, samples_types, samples_labels, samples_labels_binary = prepare_audio_set(file_paths)
samples = np.swapaxes(samples, 1, 2)

[INFO] preparing data...


In [9]:
training_samples = samples[samples_types == 0]
training_labels = samples_labels_binary[samples_types == 0]
training_subject_ids = samples_ids[samples_types == 0]

test_samples = samples[samples_types == 1]
test_labels = samples_labels_binary[samples_types == 1]

val_samples = samples[samples_types == 2]
val_labels_df = samples_labels_binary[samples_types == 2]

In [10]:
del samples, samples_ids, samples_types, samples_labels, samples_labels_binary
del file_ids, subject_ids, file_paths, types, labels, labels_binary

In [11]:
SEED = 42
NUM_CLIENTS = 8
BATCH_SIZE = 8
NUM_ROUNDS = 5
DEPRESSIVE_MULTIPLIER = 30
NON_DEPRESSIVE_MULTIPLIER = 10

In [12]:
def partition_data(X: np.ndarray, X_ids: np.ndarray, n_clients: int, d_mult: int, n_d_mult: int) -> Tuple[List[np.ndarray], List[np.ndarray]]:

    unique_subject_ids, counts = np.unique(X_ids, return_counts=True)

    # grouping the training samples by patient
    training_samples_grouped = []
    for i in unique_subject_ids:
        training_samples_grouped.append(X[X_ids == i])
    training_samples_grouped = np.array(training_samples_grouped)

    mask_30_segments = np.array([sample.shape[0] == DEPRESSIVE_MULTIPLIER for sample in training_samples_grouped])

    # creating masks to get deppressives and non deppressives
    mask_10_segments = np.array([sample.shape[0] == NON_DEPRESSIVE_MULTIPLIER for sample in training_samples_grouped])

    data_array_30_segments = training_samples_grouped[mask_30_segments]

    data_array_10_segments = training_samples_grouped[mask_10_segments]

    # recreating labels
    X_train_zeros = np.array_split(data_array_10_segments, NUM_CLIENTS)
    X_train_ones = np.array_split(data_array_30_segments, NUM_CLIENTS)

    # concatenating splitted ones and zeros with labels
    X_train_splitted = [] # (NUM_CLIENTS, data)
    y_train_splitted = [] # (NUM_CLIENTS, labels)
    for i in range(NUM_CLIENTS):

        # stack the segments from groups then concatenate
        client_data = np.concatenate((np.vstack(X_train_zeros[i]), np.vstack(X_train_ones[i])), axis=0)
        client_labels = np.concatenate((np.zeros((X_train_zeros[i].shape[0] * NON_DEPRESSIVE_MULTIPLIER), dtype=int), np.ones((X_train_ones[i].shape[0] * DEPRESSIVE_MULTIPLIER), dtype=int)), axis=0)

        X_train_splitted.append(client_data)
        y_train_splitted.append(client_labels)

    return X_train_splitted, y_train_splitted

X_trains, y_trains = partition_data(training_samples, training_subject_ids, NUM_CLIENTS, DEPRESSIVE_MULTIPLIER, NON_DEPRESSIVE_MULTIPLIER)

<ipython-input-12-45e26c792d34>:9: VisibleDeprecationWarning: Creating an ndarray from ragged nested sequences (which is a list-or-tuple of lists-or-tuples-or ndarrays with different lengths or shapes) is deprecated. If you meant to do this, you must specify 'dtype=object' when creating the ndarray.
  training_samples_grouped = np.array(training_samples_grouped)


In [ ]:
X_trains[0].shape

(610, 15001, 13)

In [14]:
def get_model(input_shape):
    model = tf.keras.models.Sequential()

    model.add(tf.keras.layers.GRU(units = 64, input_shape = input_shape)) 

    model.add(tf.keras.layers.Dense(32, activation='relu'))
    model.add(tf.keras.layers.Dropout(0.3))

    model.add(tf.keras.layers.Dense(1, activation='sigmoid'))

    return model

In [24]:
from flwr.common.typing import NDArrays
class FlowerClient(fl.client.NumPyClient):

    def __init__(self, model: tf.keras.models.Sequential, X_train: np.ndarray, y_train: np.ndarray):
        self.model = model
        self.X_train = X_train
        self.y_train = y_train

    def get_parameters(self, config):
        return self.model.get_weights()


    def fit(self, parameters: NDArrays, config: Dict[str, Scalar]) -> NDArrays:

        self.model.compile(optimizer=tf.keras.optimizers.Adam(learning_rate=0.0001), loss=tf.keras.losses.BinaryCrossentropy(label_smoothing=0.1), metrics=['accuracy'])

        self.model.set_weights(parameters)

        history = self.model.fit(self.X_train, self.y_train ,batch_size=BATCH_SIZE, epochs=1, verbose=0)
        results = {
            "loss": history.history["loss"][0],
            "accuracy": history.history["accuracy"][0],
        }
        return self.model.get_weights(), len(self.X_train), results

    def evaluate(self, parameters: NDArrays, config: Dict[str, Scalar])-> Tuple[float, int, Dict[str, Scalar]]:
        self.model.compile(optimizer=tf.keras.optimizers.Adam(learning_rate=0.0001), loss=tf.keras.losses.BinaryCrossentropy(label_smoothing=0.1), metrics=['accuracy'])
        self.model.set_weights(parameters)

        loss, acc = self.model.evaluate(self.X_train, self.y_train, verbose=0)
        return loss, len(self.X_train), {"accuracy": acc}

In [25]:
# client creator by client id
def create_client_fn(cid: str) -> FlowerClient:

    input_shape = (15001, 13)
    model = get_model(input_shape)
    cid_int = int(cid)
    return FlowerClient(model, X_trains[cid_int], y_trains[cid_int])

In [26]:
def weighted_average(metrics: List[Tuple[int, Metrics]]) -> Metrics:
    accuracies = [num_examples * m["accuracy"] for num_examples, m in metrics]
    examples = [num_examples for num_examples, _ in metrics]

    # Aggregate and return custom metric (weighted average)
    return {"accuracy": sum(accuracies) / sum(examples)}


In [27]:
patience = 5
best_accuracy = 0.0
best_loss = 999
weights = np.array([])

def evaluate(
    server_round: int,
    parameters: fl.common.NDArrays,
    config: Dict[str, fl.common.Scalar],
    ) -> Optional[Tuple[float, Dict[str, fl.common.Scalar]]]:
    """Centralized evaluation function"""

    input_shape = (15001, 13)
    model = get_model(input_shape)
    model.compile(optimizer=tf.keras.optimizers.Adam(learning_rate=0.0001), loss=tf.keras.losses.BinaryCrossentropy(label_smoothing=0.1), metrics=['accuracy'])
    model.set_weights(parameters)

    loss, accuracy = model.evaluate(val_samples, val_labels_df, batch_size=16, verbose=0)

    global best_accuracy
    global best_loss
    global weights

    print(f"LOSS: {loss}")
    print(f"BEST_LOSS: {best_loss}")
    print(f"ACCURACY: {accuracy}")
    print(f"BEST_ACCURACY: {best_accuracy}")

    if loss < best_loss:
        best_accuracy = accuracy
        weights = parameters
        best_loss = loss

    return loss, {"accuracy": accuracy}


client_resources = {"num_cpus": 2}
if tf.config.get_visible_devices("GPU"):
    client_resources["num_gpus"] = 1

# Specify the Strategy
strategy = fl.server.strategy.FedAvg(
    fraction_fit=1.0,  # Sample 100% of available clients for training
    fraction_evaluate=1.0,
    min_fit_clients=NUM_CLIENTS,
    min_evaluate_clients=NUM_CLIENTS,
    min_available_clients=NUM_CLIENTS,  # Wait until all 8 clients are available
    evaluate_metrics_aggregation_fn=weighted_average,
    evaluate_fn=evaluate
)

# Start simulation
history = fl.simulation.start_simulation(
    client_fn=create_client_fn,
    num_clients=NUM_CLIENTS,
    config=fl.server.ServerConfig(num_rounds=15),
    strategy=strategy,
    client_resources=client_resources,
)

INFO flwr 2023-12-13 08:54:23,098 | app.py:178 | Starting Flower simulation, config: ServerConfig(num_rounds=15, round_timeout=None)
INFO:flwr:Starting Flower simulation, config: ServerConfig(num_rounds=15, round_timeout=None)
2023-12-13 08:54:27,477	INFO worker.py:1621 -- Started a local Ray instance.
INFO flwr 2023-12-13 08:54:29,193 | app.py:213 | Flower VCE: Ray initialized with resources: {'memory': 32565475739.0, 'CPU': 8.0, 'object_store_memory': 16282737868.0, 'GPU': 1.0, 'node:172.28.0.12': 1.0, 'node:__internal_head__': 1.0}
INFO:flwr:Flower VCE: Ray initialized with resources: {'memory': 32565475739.0, 'CPU': 8.0, 'object_store_memory': 16282737868.0, 'GPU': 1.0, 'node:172.28.0.12': 1.0, 'node:__internal_head__': 1.0}
INFO flwr 2023-12-13 08:54:29,195 | app.py:219 | Optimize your simulation with Flower VCE: https://flower.dev/docs/framework/how-to-run-simulations.html
INFO:flwr:Optimize your simulation with Flower VCE: https://flower.dev/docs/framework/how-to-run-simulations

LOSS: 1.0393654108047485
BEST_LOSS: 999
ACCURACY: 0.550000011920929
BEST_ACCURACY: 0.0


DEBUG flwr 2023-12-13 08:57:49,769 | server.py:236 | fit_round 1 received 8 results and 0 failures
DEBUG:flwr:fit_round 1 received 8 results and 0 failures
WARNING flwr 2023-12-13 08:57:49,785 | fedavg.py:242 | No fit_metrics_aggregation_fn provided
INFO flwr 2023-12-13 08:57:59,275 | server.py:125 | fit progress: (1, 0.879640519618988, {'accuracy': 0.550000011920929}, 190.07454040500033)
INFO:flwr:fit progress: (1, 0.879640519618988, {'accuracy': 0.550000011920929}, 190.07454040500033)
DEBUG flwr 2023-12-13 08:57:59,278 | server.py:173 | evaluate_round 1: strategy sampled 8 clients (out of 8)
DEBUG:flwr:evaluate_round 1: strategy sampled 8 clients (out of 8)


LOSS: 0.879640519618988
BEST_LOSS: 1.0393654108047485
ACCURACY: 0.550000011920929
BEST_ACCURACY: 0.550000011920929


DEBUG flwr 2023-12-13 08:59:25,145 | server.py:187 | evaluate_round 1 received 8 results and 0 failures
DEBUG:flwr:evaluate_round 1 received 8 results and 0 failures
DEBUG flwr 2023-12-13 08:59:25,148 | server.py:222 | fit_round 2: strategy sampled 8 clients (out of 8)
DEBUG:flwr:fit_round 2: strategy sampled 8 clients (out of 8)
DEBUG flwr 2023-12-13 09:02:17,771 | server.py:236 | fit_round 2 received 8 results and 0 failures
DEBUG:flwr:fit_round 2 received 8 results and 0 failures
INFO flwr 2023-12-13 09:02:27,375 | server.py:125 | fit progress: (2, 0.775047779083252, {'accuracy': 0.550000011920929}, 458.17402977599977)
INFO:flwr:fit progress: (2, 0.775047779083252, {'accuracy': 0.550000011920929}, 458.17402977599977)
DEBUG flwr 2023-12-13 09:02:27,377 | server.py:173 | evaluate_round 2: strategy sampled 8 clients (out of 8)
DEBUG:flwr:evaluate_round 2: strategy sampled 8 clients (out of 8)


...

DEBUG flwr 2023-12-13 09:53:18,823 | server.py:187 | evaluate_round 13 received 8 results and 0 failures
DEBUG:flwr:evaluate_round 13 received 8 results and 0 failures
DEBUG flwr 2023-12-13 09:53:18,826 | server.py:222 | fit_round 14: strategy sampled 8 clients (out of 8)
DEBUG:flwr:fit_round 14: strategy sampled 8 clients (out of 8)
DEBUG flwr 2023-12-13 09:56:14,785 | server.py:236 | fit_round 14 received 8 results and 0 failures
DEBUG:flwr:fit_round 14 received 8 results and 0 failures
INFO flwr 2023-12-13 09:56:24,319 | server.py:125 | fit progress: (14, 0.6825131177902222, {'accuracy': 0.5475000143051147}, 3695.1184985990003)
INFO:flwr:fit progress: (14, 0.6825131177902222, {'accuracy': 0.5475000143051147}, 3695.1184985990003)
DEBUG flwr 2023-12-13 09:56:24,322 | server.py:173 | evaluate_round 14: strategy sampled 8 clients (out of 8)
DEBUG:flwr:evaluate_round 14: strategy sampled 8 clients (out of 8)


LOSS: 0.6825131177902222
BEST_LOSS: 0.6837866306304932
ACCURACY: 0.5475000143051147
BEST_ACCURACY: 0.5387499928474426


DEBUG flwr 2023-12-13 09:57:49,724 | server.py:187 | evaluate_round 14 received 8 results and 0 failures
DEBUG:flwr:evaluate_round 14 received 8 results and 0 failures
DEBUG flwr 2023-12-13 09:57:49,726 | server.py:222 | fit_round 15: strategy sampled 8 clients (out of 8)
DEBUG:flwr:fit_round 15: strategy sampled 8 clients (out of 8)
DEBUG flwr 2023-12-13 10:00:50,494 | server.py:236 | fit_round 15 received 8 results and 0 failures
DEBUG:flwr:fit_round 15 received 8 results and 0 failures
INFO flwr 2023-12-13 10:00:59,987 | server.py:125 | fit progress: (15, 0.6816101670265198, {'accuracy': 0.550000011920929}, 3970.786519063)
INFO:flwr:fit progress: (15, 0.6816101670265198, {'accuracy': 0.550000011920929}, 3970.786519063)
DEBUG flwr 2023-12-13 10:00:59,990 | server.py:173 | evaluate_round 15: strategy sampled 8 clients (out of 8)
DEBUG:flwr:evaluate_round 15: strategy sampled 8 clients (out of 8)


LOSS: 0.6816101670265198
BEST_LOSS: 0.6825131177902222
ACCURACY: 0.550000011920929
BEST_ACCURACY: 0.5475000143051147


DEBUG flwr 2023-12-13 10:02:25,701 | server.py:187 | evaluate_round 15 received 8 results and 0 failures
DEBUG:flwr:evaluate_round 15 received 8 results and 0 failures
INFO flwr 2023-12-13 10:02:25,704 | server.py:153 | FL finished in 4056.503205037001
INFO:flwr:FL finished in 4056.503205037001
INFO flwr 2023-12-13 10:02:25,709 | app.py:226 | app_fit: losses_distributed [(1, 0.9148432937074215), (2, 0.799807577183906), (3, 0.7346454168887848), (4, 0.7083110114361377), (5, 0.696900852436715), (6, 0.6919547750594768), (7, 0.6891813186888999), (8, 0.6872253430650589), (9, 0.685520889657609), (10, 0.6841349337963347), (11, 0.6828996229679026), (12, 0.6817188443021571), (13, 0.6806632714068636), (14, 0.6793856078005852), (15, 0.6780747309644172)]
INFO:flwr:app_fit: losses_distributed [(1, 0.9148432937074215), (2, 0.799807577183906), (3, 0.7346454168887848), (4, 0.7083110114361377), (5, 0.696900852436715), (6, 0.6919547750594768), (7, 0.6891813186888999), (8, 0.6872253430650589), (9, 0.68552

In [28]:
history

History (loss, distributed):
	round 1: 0.9148432937074215
	round 2: 0.799807577183906
	round 3: 0.7346454168887848
	round 4: 0.7083110114361377
	round 5: 0.696900852436715
	round 6: 0.6919547750594768
	round 7: 0.6891813186888999
	round 8: 0.6872253430650589
	round 9: 0.685520889657609
	round 10: 0.6841349337963347
	round 11: 0.6828996229679026
	round 12: 0.6817188443021571
	round 13: 0.6806632714068636
	round 14: 0.6793856078005852
	round 15: 0.6780747309644172
History (loss, centralized):
	round 0: 1.0393654108047485
	round 1: 0.879640519618988
	round 2: 0.775047779083252
	round 3: 0.7194886207580566
	round 4: 0.699170708656311
	round 5: 0.6914127469062805
	round 6: 0.6889903545379639
	round 7: 0.6879273056983948
	round 8: 0.6869351863861084
	round 9: 0.6863436102867126
	round 10: 0.6859428286552429
	round 11: 0.685799241065979
	round 12: 0.6845251321792603
	round 13: 0.6837866306304932
	round 14: 0.6825131177902222
	round 15: 0.6816101670265198
History (metrics, distributed, evaluat

In [29]:
# printing validation results
print(best_accuracy)
print(best_loss)
print(patience)
print(weights)

0.550000011920929
0.6816101670265198
5
[array([[-0.14468935, -0.15487021, -0.11520033, ..., -0.10937705,
         0.05243111,  0.05822818],
       [ 0.14861004,  0.1690577 ,  0.13361292, ...,  0.01210249,
         0.13257517,  0.09150499],
       [ 0.09970164, -0.04803832,  0.01149381, ...,  0.10679574,
         0.16116373,  0.03768284],
       ...,
       [-0.03896639, -0.16270816,  0.0356646 , ...,  0.08474819,
         0.12993914,  0.04826183],
       [ 0.10817764,  0.11533503,  0.16700113, ...,  0.11447874,
        -0.12542164,  0.02255333],
       [-0.02850386, -0.09021209, -0.00647247, ..., -0.15612717,
         0.14257371, -0.0555552 ]], dtype=float32), array([[ 0.08680966, -0.03668498,  0.02651676, ...,  0.1284284 ,
        -0.00320715,  0.08178491],
       [-0.17349042, -0.10886156,  0.06280419, ...,  0.0921331 ,
         0.02863936,  0.03896013],
       [-0.14476648,  0.09932635,  0.04586903, ...,  0.0359214 ,
        -0.06277787, -0.03039948],
       ...,
       [ 0.07741299